In [1]:
import numpy as np
from gekko import GEKKO

In [2]:
m = GEKKO(remote=False)

In [3]:
G,H = m.Array(m.Var, 2, lb=0, ub=10, integer=False)

In [4]:
m.Maximize(100*G+125*H)

In [5]:
m.Equation(3*G+6*H<=30)
m.Equation(8*G+6*H<=44)

In [6]:
m.solve(disp=False)

In [7]:
print(f'Soluzione ottimale: G = {G.value[0]:.2f} , H = {H.value[0]:.2f}')
print(f'Valore massimo: {-m.options.OBJFCNVAL:.2f}')

Soluzione ottimale: G = 2.80 , H = 3.60
Valore massimo: 730.00


In [8]:
m = GEKKO(remote=False)

x,y = m.Array(m.Var, 2, lb=0, integer=True)

m.Equations([6*x+4*y<=24,
            x+2*y<=6,
            -x+y<=1,
            y<=2])

m.Maximize(x+y)

m.options.SOLVER=1

m.solve(disp=False)

xopt = x.value[0]
yopt = y.value[0]

print(f'x: {xopt} y: {yopt} obj: {-m.options.OBJFCNVAL}')

x: 2.0 y: 2.0 obj: 4.0


minimize
    
            xy^2 - x^2 - y^2

subject to 
 
             x + y => 4
              
             xy <= 5
              
             1 <= x <= 3
              
             2 <= y <= 3

In [9]:
m = GEKKO(remote=False)

X = m.Array(m.Var, 2, lb=1, ub=30, integer=False)
# y = m.Array(m.Var, 1, lb=1, ub=3, integer=True)
# y.LOWER = 2

In [10]:
for x in (X):
    x.UPPER = 3

In [11]:
x,y = X

y.LOWER = 2

In [12]:
m.Minimize(x*y**2-x**2-y**2)

In [13]:
m.Equations([x+y>=4,x*y<=5])

In [14]:
m.solve(disp=False)

In [15]:
xopt = x.value[0]
yopt = y.value[0]

print(f'x: {xopt} y: {yopt} obj: {-m.options.OBJFCNVAL}')

x: 1.0 y: 3.0 obj: 1.0


In [16]:
# Object function
c = [-1, 4] # -x +4y

In [17]:
# Constraints
A_u = np.array([[3,2],[1,5]])
b_u = np.array([10, 10])

In [18]:
x = m.Array(m.Var, 2, lb=0, ub=10, integer=True)

In [19]:
m.Minimize(c@x)

In [20]:
c@x

(((-1)*(int_v3))+((4)*(int_v4)))

In [21]:
x[0].name

'int_v3'

In [22]:
A_u@x

array([(((3)*(int_v3))+((2)*(int_v4))), (((1)*(int_v3))+((5)*(int_v4)))],
      dtype=object)

In [23]:
Ax = A_u@x

In [24]:
m.Equations([Ax[i]<=b_u[i] for i in range(len(b_u))])

In [25]:
m.options.SOLVER=1

In [26]:
m.solve(disp=False)

In [27]:
xopt, yopt = x[0].value[0], x[1].value[0]

print(f'x: {xopt} y: {yopt} obj: {-m.options.OBJFCNVAL}')

x: 3.0 y: 0.0 obj: 4.0


## Global optimization

minimize = 100 -x1^2 ..........

In [30]:
from hyperopt import fmin, tpe, hp
from hyperopt import STATUS_OK, STATUS_FAIL

In [29]:
# hyperopt.hp.quniform, hyperopt.hp.qloguniform
# hyperopt.hp.choice
# hyperopt.hp.randint

In [48]:
# hyperparameters space
space = {
    
    'x1':hp.quniform('x1', 0, 10, 3),
    'x2':hp.quniform('x2', 0, 10, 3),
    'x3':hp.quniform('x3', 0, 10, 3),
}

In [80]:
def objective(params):
    
    m = GEKKO(remote=False)
    
    x = m.Array(m.Var, 3, lb=0, integer=True)
    x1,x2,x3 = x
    
    x1.value = params['x1']
    x2.value = params['x2']
    x3.value = params['x3']

    m.Minimize(1000-x1**2+(1/x2**2))
    
    m.Equations([x1>=0, x2>0, x3-x2>=1])
    
    m.options.SOLVER=1
    
    m.solve(disp=False, debug=False)
    
    obj = m.options.OBJFCNVAL
    
    if m.options.APPSTATUS==1:
        s=STATUS_OK
    else:
        s=STATUS_FAIL
        
    m.cleanup()
    
    return {'loss':obj, 'status':s, 'x':x}

In [81]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=50)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 22.33trial/s, best loss: 919.01234568]


In [82]:
print(best)

{'x1': 0.0, 'x2': 9.0, 'x3': 0.0}


In [83]:
sol = objective(best)

In [84]:
print(sol)

{'loss': 1000.0002778, 'status': 'ok', 'x': array([[0.0], [60.0], [61.0]], dtype=object)}


In [88]:
0**2+(1/(60**2))

0.0002777777777777778